In [9]:
# -*- coding = utf-8 -*-
# @time:08/07/2023 00:49
# Author:Yunbo Long
# @File:data_processing.py
# @Software:PyCharm
# -*- coding = utf-8 -*-
# @time:03/07/2023 13:11
# Author:Yunbo Long
# @File:CNN_local.py
# @Software:PyCharm
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression,ElasticNet, Lasso
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingRegressor,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

from sklearn.model_selection import train_test_split
import warnings
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns


# Hiding the warnings
warnings.filterwarnings('ignore')

# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# Load dataset
# dataset = pd.read_csv('/content/drive/My Drive/DataCoSupplyChainDataset.csv')

dataset =pd.read_csv('/Users/yunbo/Documents/GitHub/PFL_Optimiozation/Data/DataCoSupplyChainDataset.csv',encoding='iso-8859-1')
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# print(dataset.head(30))


dataset['Customer Full Name'] = dataset['Customer Fname'].astype(str) + dataset['Customer Lname'].astype(str)
dataset['TotalPrice'] = dataset['Order Item Quantity'] * dataset[
    'Sales per customer']  # Multiplying item price * Order quantity



data = dataset.drop(
    ['Customer Email', 'Customer Id', 'Customer Password', 'Customer Fname', 'Customer Lname',
      'Product Description', 'Product Image', 'Order Zipcode','Product Status','Order Profit Per Order','Product Price'], axis=1)

data['Customer Zipcode'] = data['Customer Zipcode'].fillna(0)  # Filling NaN columns with zero



data['order_year'] = pd.DatetimeIndex(data['order date (DateOrders)']).year
data['order_month'] = pd.DatetimeIndex(data['order date (DateOrders)']).month
data['order_week_day'] = pd.DatetimeIndex(data['order date (DateOrders)']).day_name()
data['order_hour'] = pd.DatetimeIndex(data['order date (DateOrders)']).hour
# data['order_second'] = pd.DatetimeIndex(data['order date (DateOrders)'])

data['shipping_year'] = pd.DatetimeIndex(data['shipping date (DateOrders)']).year
data['shipping_month'] = pd.DatetimeIndex(data['shipping date (DateOrders)']).month
data['shipping_week_day'] = pd.DatetimeIndex(data['shipping date (DateOrders)']).day_name()
data['shipping_hour'] = pd.DatetimeIndex(data['shipping date (DateOrders)']).hour


label_data = data[['Order Region','shipping_week_day','order_week_day','Customer Full Name','Type','Delivery Status','Category Name','Customer City','Customer Country','Customer Segment','Customer State','Customer Street','Department Name','Market','Order City','Order Country','order date (DateOrders)','Order State','Order Status','Product Name','shipping date (DateOrders)','Shipping Mode']]

data['index'] = data['Order Region']
#


target=data['Sales']

data=data.drop(columns=['shipping_week_day','order_week_day','Customer Full Name','Sales','Type','Delivery Status','Category Name','Customer City','Customer Country','Customer Segment','Customer State','Customer Street','Department Name','Market','Order City','Order Country','order date (DateOrders)','Order State','Order Status','Product Name','shipping date (DateOrders)','Shipping Mode','Order Region','Sales per customer'])


def Labelencoder_feature(x):
    le=LabelEncoder()
    x=le.fit_transform(x)
    return x

# Exclude datetime columns from label encoding
# datetime_columns = ['order date (DateOrders)', 'shipping date (DateOrders)']
# data_encoded = data.drop(datetime_columns, axis=1)

# Apply label encoding to remaining columns
data_encoded = label_data.apply(Labelencoder_feature)

data = pd.concat([data_encoded, data], axis=1)


# # Calculate correlation matrix
# correlation_matrix = data.corr()

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# print(data.head(100))
# print(data.shape)

# Create heatmap
# plt.figure(figsize=(100, 50))
# sns.heatmap(correlation_matrix, annot=True, fmt=".3f", cmap="BuPu")
# plt.show()


data_final=data[['Order Id', 'Order Customer Id', 'Order Item Id',
 'Order Item Product Price' ,'Department Id', 'Order Item Quantity',
 'Category Id' ,'shipping_month' ,'Benefit per order' ,'Order Item Total',
 'Product Card Id', 'Product Name', 'Order Item Cardprod Id' ,
 'Order State', 'Product Category Id', 'order_week_day', 'shipping_year',
 'Category Name', 'order_month', 'order_year' ,'Order Item Discount',
 'Department Name', 'Market', 'Order City',
 'Days for shipment (scheduled)' ,'Customer Segment', 'Customer Full Name','index']]

train_data = data_final
xs=train_data.loc[:, train_data.columns != 'Sales']
ys=target
xs_train, xs_test,ys_train,ys_test = train_test_split(xs,ys,test_size = 0.3, random_state = 42)

# Concatenate train_data and ys
train_data['Sales'] = ys.values

# Save the integrated DataFrame to CSV
train_data.to_csv('integrated_train_data_ISMM_test.csv', index=False)

In [16]:
# -*- coding = utf-8 -*-
# @time:08/07/2023 00:49
# Author:Yunbo Long
# @File:data_processing.py
# @Software:PyCharm
# -*- coding = utf-8 -*-
# @time:03/07/2023 13:11
# Author:Yunbo Long
# @File:CNN_local.py
# @Software:PyCharm
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
from sklearn import svm,metrics,tree,preprocessing,linear_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression,ElasticNet, Lasso
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingRegressor,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import accuracy_score,mean_squared_error,recall_score,confusion_matrix,f1_score,roc_curve, auc
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

from sklearn.model_selection import train_test_split
import warnings
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns


# Hiding the warnings
warnings.filterwarnings('ignore')

# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# Load dataset
# dataset = pd.read_csv('/content/drive/My Drive/DataCoSupplyChainDataset.csv')

dataset =pd.read_csv('/Users/yunbo/Documents/GitHub/PFL_Optimiozation/Data/DataCoSupplyChainDataset.csv',encoding='iso-8859-1')
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# print(dataset.head(30))


dataset['Customer Full Name'] = dataset['Customer Fname'].astype(str) + dataset['Customer Lname'].astype(str)
dataset['TotalPrice'] = dataset['Order Item Quantity'] * dataset[
    'Sales per customer']  # Multiplying item price * Order quantity



data = dataset.drop(
    ['Customer Email', 'Customer Id', 'Customer Password', 'Customer Fname', 'Customer Lname',
      'Product Description', 'Product Image', 'Order Zipcode','Product Status','Order Profit Per Order','Product Price','TotalPrice'], axis=1)

data['Customer Zipcode'] = data['Customer Zipcode'].fillna(0)  # Filling NaN columns with zero



data['order_year'] = pd.DatetimeIndex(data['order date (DateOrders)']).year
data['order_month'] = pd.DatetimeIndex(data['order date (DateOrders)']).month
data['order_week_day'] = pd.DatetimeIndex(data['order date (DateOrders)']).day_name()
data['order_hour'] = pd.DatetimeIndex(data['order date (DateOrders)']).hour
# data['order_second'] = pd.DatetimeIndex(data['order date (DateOrders)'])

data['shipping_year'] = pd.DatetimeIndex(data['shipping date (DateOrders)']).year
data['shipping_month'] = pd.DatetimeIndex(data['shipping date (DateOrders)']).month
data['shipping_week_day'] = pd.DatetimeIndex(data['shipping date (DateOrders)']).day_name()
data['shipping_hour'] = pd.DatetimeIndex(data['shipping date (DateOrders)']).hour


label_data = data[['shipping_week_day','order_week_day','Customer Full Name','Type','Delivery Status','Category Name','Customer City','Customer Country','Customer Segment','Customer State','Customer Street','Department Name','Market','Order City','Order Country','Order State','Order Status','Product Name','Shipping Mode']]

data['index'] = data['Order Region']
#


target=data['Sales']

data=data.drop(columns=['shipping_week_day','order_week_day','Customer Full Name','Sales','Type','Delivery Status','Category Name','Customer City','Customer Country','Customer Segment','Customer State','Customer Street','Department Name','Market','Order City','Order Country','order date (DateOrders)','Order State','Order Status','Product Name','shipping date (DateOrders)','Shipping Mode','Order Region','Sales per customer','Order Region', 'order date (DateOrders)','shipping date (DateOrders)'])


def Labelencoder_feature(x):
    le=LabelEncoder()
    x=le.fit_transform(x)
    return x

# Exclude datetime columns from label encoding
# datetime_columns = ['order date (DateOrders)', 'shipping date (DateOrders)']
# data_encoded = data.drop(datetime_columns, axis=1)

# Apply label encoding to remaining columns
data_encoded = label_data.apply(Labelencoder_feature)

data = pd.concat([data_encoded, data], axis=1)


# # Calculate correlation matrix
# correlation_matrix = data.corr()

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# print(data.head(100))
# print(data.shape)

# Create heatmap
# plt.figure(figsize=(100, 50))
# sns.heatmap(correlation_matrix, annot=True, fmt=".3f", cmap="BuPu")
# plt.show()


# data_final=data[['Order Id', 'Order Customer Id', 'Order Item Id',
#  'Order Item Product Price' ,'Department Id', 'Order Item Quantity',
#  'Category Id' ,'shipping_month' ,'Benefit per order' ,'Order Item Total',
#  'Product Card Id', 'Product Name', 'Order Item Cardprod Id' ,
#  'Order State', 'Product Category Id', 'order_week_day', 'shipping_year',
#  'Category Name', 'order_month', 'order_year' ,'Order Item Discount',
#  'Department Name', 'Market', 'TotalPrice', 'Order City',
#  'Days for shipment (scheduled)' ,'Customer Segment', 'Customer Full Name','index']]
data = data.drop(columns=['index'])
data

,shipping_week_day,order_week_day,Customer Full Name,Type,Delivery Status,Category Name,Customer City,Customer Country,Customer Segment,Customer State,...,Order Item Quantity,Order Item Total,Product Card Id,Product Category Id,order_year,order_month,order_hour,shipping_year,shipping_month,shipping_hour
0,2,6,1876,1,0,40,66,1,0,36,...,1,314.640015,1360,73,2018,1,22,2018,2,22
1,4,2,5378,3,1,40,66,1,0,36,...,1,311.359985,1360,73,2018,1,12,2018,1,12
2,6,2,4429,0,3,40,452,0,0,5,...,1,309.720001,1360,73,2018,1,12,2018,1,12
3,5,2,12929,1,0,40,285,0,2,5,...,1,304.809998,1360,73,2018,1,11,2018,1,11
4,1,2,10638,2,0,40,66,1,1,36,...,1,298.250000,1360,73,2018,1,11,2018,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180514,6,2,8426,0,3,18,59,0,2,31,...,1,399.980011,1004,45,2016,1,3,2016,1,3
180515,5,2,11679,1,1,18,26,0,1,5,...,1,395.980011,1004,45,2016,1,1,2016,1,1
180516,6,0,6300,3,1,18,55,0,1,7,...,1,391.980011,1004,45,2016,1,21,2016,1,21
180517,1,0,9384,2,0,18,66,1,0,36,...,1,387.980011,1004,45,2016,1,20,2016,1,20


In [17]:

#Feature Selection based on importance
from sklearn.feature_selection import f_regression
F_values, p_values  = f_regression(data, target)

import itertools
f_reg_results = [(i, v, z) for i, v, z in itertools.zip_longest(data.columns, F_values,  ['%.3f' % p for p in p_values])]
f_reg_results=pd.DataFrame(f_reg_results, columns=['Variable','F_Value', 'P_Value'])

f_reg_results=pd.DataFrame(f_reg_results, columns=['Variable','F_Value', 'P_Value'])
f_reg_results = f_reg_results.sort_values(by=['P_Value'])
f_reg_results.P_Value= f_reg_results.P_Value.astype(float)
f_reg_results=f_reg_results[f_reg_results.P_Value<0.06]
print(f_reg_results)

f_reg_list=f_reg_results.Variable.values
print(f_reg_list)

                         Variable       F_Value  P_Value
33                  Order Item Id  1.133744e+03    0.000
25                  Department Id  1.052499e+04    0.000
37               Order Item Total  8.665299e+06    0.000
23                    Category Id  9.851295e+03    0.000
44                 shipping_month  5.764518e+02    0.000
21              Benefit per order  3.192008e+03    0.000
38                Product Card Id  1.091556e+04    0.000
17                   Product Name  3.775172e+04    0.000
31            Order Item Discount  1.112179e+05    0.000
15                    Order State  2.792909e+01    0.000
28              Order Customer Id  6.733401e+02    0.000
36            Order Item Quantity  2.068679e+03    0.000
34       Order Item Product Price  2.996046e+05    0.000
11                Department Name  5.240946e+02    0.000
39            Product Category Id  9.851295e+03    0.000
29                       Order Id  1.157925e+03    0.000
40                     order_ye

In [ ]:
['Order Item Id' 'Department Id' 'Order Item Total' 'Category Id'
 'shipping_month' 'Benefit per order' 'Product Card Id' 'Product Name'
 'Order Item Discount' 'Order State' 'Order Customer Id'
 'Order Item Quantity' 'Order Item Product Price' 'Department Name'
 'Product Category Id' 'Order Id' 'order_year' 'order_month'
 'Category Name' 'Order Item Cardprod Id' 'shipping_year' 'order_week_day'
 'Market' 'Order City' 'Days for shipment (scheduled)' 'Customer Segment'
 'Customer Full Name']

['Order Item Id' 'Department Id' 'Order Item Total' 'Category Id'
 'shipping_month' 'Benefit per order' 'Product Card Id' 'Product Name'
 'Order Item Discount' 'Order State' 'Order Customer Id'
 'Order Item Quantity' 'Order Item Product Price' 'Department Name'
 'Product Category Id' 'Order Id' 'order_year' 'order_month'
 'Category Name' 'Order Item Cardprod Id' 'shipping_year' 'order_week_day'
 'Market' 'Order City' 'Days for shipment (scheduled)' 'Customer Segment'
 'Customer Full Name']

data_final=data[['Order Id', 'Order Customer Id', 'Order Item Id',
 'Order Item Product Price' ,'Department Id', 'Order Item Quantity',
 'Category Id' ,'shipping_month' ,'Benefit per order' ,'Order Item Total',
 'Product Card Id', 'Product Name', 'Order Item Cardprod Id' ,
 'Order State', 'Product Category Id', 'order_week_day', 'shipping_year',
 'Category Name', 'order_month', 'order_year' ,'Order Item Discount',
 'Department Name', 'Market', 'Order City',
 'Days for shipment (scheduled)' ,'Customer Segment', 'Customer Full Name','index']]